In [0]:
%run ../source_to_bronze/utils


In [0]:
silver_path = "/Volumes/assignment/silver/silverdata/silver"

employee_df = spark.read.format("delta").load(f"{silver_path}/dim_employee")
department_df = spark.read.format("delta").load(f"{silver_path}/dim_department")
country_df = spark.read.format("delta").load(f"{silver_path}/dim_country")

display(employee_df)


# salary by descending

In [0]:
salary_by_department = employee_df.groupBy("department").sum("salary").orderBy("sum(salary)", ascending=False)


# Employee count by department and country

In [0]:
employee_location_count = employee_df.groupBy("department", "country").count()


# List department names with country names (join with lookup tables)

In [0]:
dept_country_join = employee_df \
    .join(department_df, employee_df.department == department_df.department_id, "left") \
    .join(country_df, employee_df.country == country_df.country_code, "left") \
    .select("employee_df.employee_id", "department_df.department_name", "country_df.country_name")


# avg_age_department = employee_df.groupBy("department").avg("age")


In [0]:
avg_age_department = employee_df.groupBy("department").avg("age")


In [0]:
from pyspark.sql.functions import current_date

fact_employee_df = employee_df.withColumn("at_load_date", current_date())


In [0]:
gold_path = "/Volumes/assignment/gold/golddata/gold"

dbutils.fs.mkdirs(gold_path)

fact_employee_df.write.format("delta").mode("overwrite").save(f"{gold_path}/fact_employee")


In [0]:
display(dbutils.fs.ls(gold_path))


In [0]:
fact_employee_df = spark.read.format("delta").load(f"{gold_path}/fact_employee")
display(fact_employee_df)


In [0]:
salary_desc = fact_employee_df.groupBy("department").sum("salary").orderBy("sum(salary)", ascending=False)
display(salary_desc)


In [0]:
emp_count = fact_employee_df.groupBy("department", "country").count()
display(emp_count)


In [0]:
dept_country = fact_employee_df.select("department", "country")
display(dept_country.distinct())


In [0]:
avg_age = fact_employee_df.groupBy("department").avg("age")
display(avg_age)
